### Analysis Plan

1) How many are '.'s in each year in the fte file?  
2) What are the average and sum FTE for each department and program.  
3) Do the department names from the two files match?  
4) Make dates comparable between both files.  
5) Drop unnecessary dates from ssa.  
6) Is there are correlation between FTE and number of indeterminate employees? 

## Import packages and files

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import csv

# Run "pip install numpy" in a terminal to get the numpy package
# You may or may not have to do the same thing with the datetime package.
 # Try to import it and see whether or not you get an error

fte = pd.read_csv('ftebp-eng.csv')
ssa = pd.read_csv('ssa-pop1-eng.csv')

## 1) How many are '.'s in each year in the fte file?

### Introducing list slicing

In [ ]:
a_list = [1,2,3,4,5]
print(a_list[2:])
print(a_list[3:4])
print(a_list[3:-1])

### Using slicing to loop over specific columns

In [ ]:
for column in fte.columns[2:]:
    dot_count = len(fte[fte[column] == '.'])
    print(column, dot_count)

## 2) What are the average and sum FTE for each department and program. 

In [ ]:
# first, convert values to floats. Can't do math on strings.
for column in fte.columns[2:]:
    fte[column] = fte[column].replace(".", "0")
    fte[column] = fte[column].astype(float)

In [ ]:
fte.groupby(['Department']).sum()

In [ ]:
fte.groupby(['Department']).mean()

## 3) Do the department names from the two files match?

In [ ]:
# get unique lists of department names for each file
ssa_dept_names = ssa['Departments and Agencies'].unique()
fte_dept_names = fte['Department'].unique()

In [ ]:
# find names in fte that are not in ssa
in_fte_not_ssa = np.setdiff1d(fte_dept_names, ssa_dept_names)

# find names in ssa that are not in fte
in_ssa_not_fte = np.setdiff1d(ssa_dept_names, fte_dept_names)

print(f"There are {len(in_fte_not_ssa)} names in fte that are not in ssa, and {len(in_ssa_not_fte)} names that are in ssa that aren't in fte")

### Making the names compatible

In [ ]:
# create a dict that can convert fte names to ssa names
fte_to_ssa = {}
with open('fte_ssa_equivalents.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        fte_to_ssa[row[0]] = row[1]

In [ ]:
# convert names in fte to their ssa equivalent
# remember, many fte names have no equivalent, and vice versa
def fte_dept_to_ssa(department_name):
    try:
        dept_2 = fte_to_ssa[department_name]
    except:
        dept_2 = None
    return dept_2

fte['dept_2'] = fte.apply(lambda x: fte_dept_to_ssa(x['Department']), axis=1)
fte = fte[fte['dept_2'].notnull()]
fte = fte.drop(['Department', 'Program'], axis=1)
# fte now only contains those department for which there was an ssa equivalent.
# that is not true of the ssa file.

### Before we proceed, let's reshape the fte data

In [ ]:
fte = fte.groupby(['dept_2']).sum()

In [ ]:
fte = fte.reset_index()

In [ ]:
fte = fte.melt('dept_2', value_name="FTE")

In [ ]:
fte = fte.rename({"variable": "fy"}, axis='columns')

## 4) Make dates comparable between both files.

### Introducing datetime
refer to this: https://www.programiz.com/python-programming/datetime/strptime  

In [ ]:
# strptime: string parse time (from a string to a datetime object)
# strftime: string format time (from a datetime object to a formatted string)
sample_date = datetime.strptime("201003","%Y%m")
print(sample_date.strftime('%y-%m-%d'))
print(sample_date.year)

In [ ]:
# datetime inequality example
d1 = datetime.strptime('201104',"%Y%m")
d2 = datetime.strptime('201603',"%Y%m")
d1 < d2

### Back to modifying dates in ssa

In [ ]:
# store datetime objects in the dataframe
ssa['new_date'] = ssa.apply(lambda x: datetime.strptime(str(x['Date']),"%Y%m"), axis=1)

## 5) Drop unnecessary dates from ssa

In [ ]:
def keep_dates_within_range(row):    
    min_date = datetime.strptime('201104',"%Y%m")
    max_date = datetime.strptime('201603',"%Y%m")
    row_date = row['new_date']
    if row_date >= min_date and row_date <= max_date:
        return True
    else:
        return False
    
ssa = ssa[ssa.apply(keep_dates_within_range,axis=1)]

#TODO: can we return row or None instead of true/false

### Add fiscal year column to ssa

In [ ]:
# to understand SettingWithCopyWarning, read top answer here:
# https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
# it can be ignored here.

def determine_fy(year_and_month):
    year_and_month = str(year_and_month)
    date = datetime.strptime(year_and_month,"%Y%m")
    y2 = int(str(date.year)[2:])
    if date.month >= 4:
        fy = f"FTE 20{y2}-{y2+1}"
    else:
        fy = f"FTE 20{y2-1}-{y2}"
    return fy

ssa['fy'] = ssa.apply(lambda x: determine_fy(x['Date']), axis=1)

## 6) Is there are correlation between FTE and number of indeterminate employees? 

### Creating a lookup function for indeterminate employees in ssa

In [ ]:
def get_indeterminate_count(fy, department):
    row = ssa[(ssa['Departments and Agencies'] == department)
              &(ssa['fy'] == fy)
              &(ssa['Tenure'] == "Indeterminate")]
    indeterminate_count = row['Employees'].sum()
    return indeterminate_count

#get_indeterminate_count('FTE 2015-16', "Veterans Affairs Canada")
fte['indeterminate'] = fte.apply(lambda x: get_indeterminate_count(x['fy'],x['dept_2']), axis=1)

## Figuring out how to test correlation is up to you
### use google